In [108]:
import tkinter as tk
from tkinter import ttk
import numpy as np

from ipywidgets import interact, interact_manual
import ipywidgets as widgets

In [101]:
def main(game_size):
    # Create the entire GUI program
    custom = np.array([[0, 2, 4, 8],[16, 32, 64, 128],[256, 512, 1024, 2048],[4096, 8192, 16384, 32768]])
    #program = ej2048(4, custom_start=True, custom_board=custom)
    
    program = evan2048(game_size)
    
    # Start the GUI event loop
    program.window.mainloop()


class evan2048:
   
    def __init__(self, dimension, custom_start=False, custom_board=np.zeros((4, 4), dtype=int)):
        

        self.custom_start = custom_start
        self.custom_board = custom_board
        
        # Initialize main window
        self.window = tk.Tk()
        
        self.window.grid_rowconfigure(0, weight=0)
        self.window.grid_columnconfigure(0, weight=1)
        self.window.grid_rowconfigure(1, weight=1)
        

        self.window.title('Evan\'s 2048')
        
        
        # main window key bindings
        self.window.bind('<KeyPress>', self.key_press)
        
        # Initialize tile color dictionary
        self.tile_color_dict = {'mainBg': '#bbada0', 'darktext': '#776e65', 'lighttext': '#f9f6f2',
            0: '#ccc0b4', 2: '#eee4da', 4: '#ede0c8', 8: '#f2b179', 16: '#f59563',
            32: '#f67c5f', 64: '#f65e3b', 128: '#edcf72', 256: '#edcc61',
            512: '#edc850', 1024: '#edc53f', 2048: '#edc22e', 4096: '#ed702e',
            8192: '#ed4c2e', 16384: '#c83d22', 32768: '#A0301B', 65536: '#802615'}
        
        
        # Initialize game variables
        self.dim = dimension
        self.score = 0
        self.new_game_board()
        
        # Initialize header
        self.build_header()
        
        # Initialize game board
        self.build_game_board()
        
        #update window minimum size
        self.window.update()
        self.window.minsize(self.window.winfo_width(), self.window.winfo_height())
  
    ####################################################################################
    ####################################################################################
    # GUI methods:
    
    # Method to build the header
    def build_header(self):
        
        # Initialize header dictionary
        self.header = {}
        
        # Initialize header frame
        self.header['frame'] = tk.Frame(self.window)
        self.header['frame'].grid(row=0, column=0, sticky=tk.W+tk.E)
        self.header['frame'].grid_columnconfigure(0, weight=1)

        frame_row = 0
        # Initialize header label
        self.header['game label'] = tk.Label(self.header['frame'], text='Welcome to Evan\'s 2048 game!', bg='white')
        self.header['game label'].grid(row=frame_row, column=0, columnspan=self.dim, sticky=tk.W+tk.E, padx=10, pady=5)
        frame_row += 1
        
        # Initialize score label
        self.header['score label'] = tk.Label(self.header['frame'], text='Score: {}'.format(self.score), bg='white')
        self.header['score label'].grid(row=frame_row, column=self.dim-1, sticky=tk.E, padx=10, pady=5)
        frame_row += 1
        
        # Initialize game over label
        self.header['game over label'] = tk.Label(self.header['frame'], 
                                                  text='Game Over: {}'.format(self.game_over(self.board)), 
                                                  bg='white')
        self.header['game over label'].grid(row=frame_row, column=self.dim-1, sticky=tk.E, padx=10, pady=5)
        frame_row += 1
        
        # Initialize settings button (to add later)
        #self.header['settings button'] = ttk.Button(self.header['frame'], text="Settings", command=self.create_settings_window)
        #self.header['settings button'].grid(row=3, column=0, sticky=tk.W, padx=10, pady=5)
        
       
        # Initialize new game button
        self.header['new game button'] = ttk.Button(self.header['frame'], text='New Game!', takefocus=False)
        self.header['new game button'].grid(row=frame_row, column=self.dim-1, sticky=tk.E, padx=10, pady=5)
        self.header['new game button']['command'] = self.new_2048
        
        
    # Method to build the game board
    def build_game_board(self):
        
        # Initialize game board dictionary
        self.game_board = {}
        
        # Initialize game board frame
        self.game_board['frame'] = tk.Frame(self.window)
        self.game_board['frame'].grid(row=1, column=0, sticky=tk.N+tk.S+tk.W+tk.E)
        
        # Initialize game board tiles
        self.game_board['tiles'] = self.create_tiles()
        
    def create_tiles(self):
        tile_array = []
        
        for i in range(self.dim): # Loop over rows
            tile_row = []
       
            for j in range(self.dim): # Loop over columns
                value = self.board[i][j]
                color = self.tile_color_dict[value]
                # Handle '0'-tiles as blank
                if value == 0:
                    value = ''
                label = tk.Label(master=self.game_board['frame'], text='{}'.format(value), bg=color, fg='white')
                label.config(font=('Helvetica', 24))
                label.grid(row=i, column=j, padx=1, pady=1, sticky=tk.N+tk.S+tk.W+tk.E)
                tile_row.append(label)
            
            tile_array.append(tile_row)
            
            self.game_board['frame'].columnconfigure(i, weight=1, minsize=75)
            self.game_board['frame'].rowconfigure(i, weight=1, minsize=75)
        
        return tile_array
        
    # Method for updating tile label values
    def update_tiles(self):
        #print('board from update_tiles:\n{}'.format(board))
        for i in range(self.dim):
           
            for j in range(self.dim):
                value = self.board[i][j]
                color = self.tile_color_dict[value]
                # Handle '0'-tiles as blank
                if value == 0:
                    value = ''
                
                #self.tile_array[i][j].config(text='{}'.format(value), bg=color)
                self.game_board['tiles'][i][j].config(text='{}'.format(value), bg=color)
    
    # Method for updating score label
    def update_score(self):
        #self.score_label.config(text='Score: {}'.format(self.score))
        self.header['score label'].config(text='Score: {}'.format(self.score))
    
    # Method for updating game over label
    def update_game_over(self):
        #self.game_over_label.config(text='Game Over: {}'.format(self.game_over(self.board)))
        self.header['game over label'].config(text='Game Over: {}'.format(self.game_over(self.board)))
                
    
    def key_press(self, key):
        #print('board from key_press:\n{}'.format(self.board))
        if key.keysym not in ['Left', 'Right', 'Up','Down']:
            print('Please press a valid key: {}'.format(key.keysym))
            
        else:    # Get direction and merge board
            #print('A valid key was pressed: {}'.format(key.keysym))
            if key.keysym == 'Up':
                self.key_up(key)
            elif key.keysym == 'Down':
                self.key_down(key)
            elif key.keysym == 'Left':
                self.key_left(key)
            elif key.keysym == 'Right':
                self.key_right(key)
        
            # call update methods
            
            self.update_tiles()
            self.update_score()
            self.update_game_over()
            
            #if self.game_over == True:
                
            
    # Method for handling up merge key
    def key_up(self, key):
        #print('key_up: Merge {}!'.format(key.keysym.lower()))
        dir='up'
        temp_board = self.board
        self.board = self.merge_board(dir, temp_board)

    # Method for handling down merge key
    def key_down(self, key):
        #print('key_down: Merge {}!'.format(key.keysym.lower()))
        dir='down'
        temp_board = self.board
        self.board = self.merge_board(dir, temp_board)

    # Method for handling left merge key
    def key_left(self, key):
        #print('key_left: Merge {}!'.format(key.keysym.lower()))
        dir='left'
        #print('board from key_left:\n{}'.format(self.board))
        temp_board = self.board
        self.board = self.merge_board(dir, temp_board)

    # Method for handling right merge key        
    def key_right(self, key):
        #print('key_right: Merge {}!'.format(key.keysym.lower()))
        dir='right'
        temp_board = self.board
        self.board = self.merge_board(dir, temp_board)
    
    ####################################################################################
    ####################################################################################
    # Game logic methods:
    
    # Method for moving all non-zero tiles in a single row to the left
    def slide_row_left(self, row):
        
        mask = np.isin(row, 0, invert=True)
        nonzeros = row[mask]
        zeros = np.zeros(len(row)-len(nonzeros), dtype=int)
        slide_row = np.append(nonzeros, zeros)
        return slide_row
    
    # Method for merging identical adjacent tiles in a single row into a single tile with the sum as its value.
    # Note, this method should only merge pairs once, i.e. [2, 2, 2, 2] -> [4, 4, 0, 0] and not [8, 0, 0, 0].
    # The score is incremented by the new combined tile's value when a merge occurs.
    def merge_row_left(self, row):
    
        temp_row = row
        new_row = []
        
        # Slide all non-zero elements to the left:
        temp_row = self.slide_row_left(row)
    
        # Loop over pairs while temp_row at least two elements
        # Check if the first two elements in temp_row are equal.
        # If equal: append their merged value to new_row. drop first two elements of temp_row
        # If not equal: append first value to new_row. drop first element of temp_row
        while len(temp_row) >= 2:

            if temp_row[0] == temp_row[1]:
                new_row.append(temp_row[0] + temp_row[1])
                self.score += temp_row[0] + temp_row[1]
                temp_row = temp_row[2:]

            else:
                new_row.append(temp_row[0])
                temp_row = temp_row[1:]
    
        # Treat last pair separately
        if len(temp_row) == 2:
            if row[-2] == row[-1]:
                new_row.append(row[-2] + row[-1])
                new_row.append(0)
                self.score += row[-2] + row[-1]
            else:
                new_row.append(row[-2])
                new_row.append(row[-1])
        elif len(temp_row) == 1:
            new_row.append(temp_row[-1])
        
        return new_row + [0]*(len(row) - len(new_row))
    
    
    # Method to merge the entire board to the left by looping over the board rows
    def merge_board_left(self, board):

        temp_board_merge = []
        
        # Merge all rows left:
        for row_index in range(self.dim):
            row = board[row_index]
            merged_row = self.merge_row_left(row)
            temp_board_merge.append(merged_row)
        temp_board_merge = np.array(temp_board_merge, dtype=int)

        #print('board from merge_board_left:\n{}'.format(temp_board_merge))
        return np.array(temp_board_merge)
    
    # Method for generating a new non-zero tile after merge:
    def new_tile(self, board):
        
        # Check if there are any zero tiles. If none, return -1
        mask = np.isin(board, 0)
        if mask.any() == False:
            return board, -1
        
        # Get coordinates of zero tiles
        coords = np.transpose(np.nonzero(mask))
        # Choose a random zero tile
        index = np.random.choice(len(coords))
        # Add two to the chosen tile
        board[coords[index][0]][coords[index][1]] += 2
        
        return board, 0
    
    # Method for checking if the game is over
    def game_over(self, board):
        
        # Return False if there are any empty tiles
        if np.isin(board, 0).any() == True:
            return False
        
        # Return False if there are any available merges by checking adjacent pairs in rows and columns
        # board for testing if any available left or right merges
        test_left_right = board
        # board for testing if any up or down merges
        test_up_down = np.rot90(board)
        
        # loop over rows
        for i in range(len(board)):
            # loop over columns except for last to test adjacent pairs
            for j in range(len(board[i])-1):
                if test_left_right[i][j] == test_left_right[i][j+1]:
                    return False
                if test_up_down[i][j] == test_up_down[i][j+1]:
                    return False            
        
        # Return True if no empty tiles or no available merges
        return True
    
    # Method for merging board in any direction
    def merge_board(self, dir, board):

        board_before_merge = board
        
        # Perform merge in chosen direction
        if dir == 'left':
            temp_board = self.merge_board_left(board_before_merge)
            
        elif dir == 'right':
            temp_board = np.fliplr(board_before_merge)    
            temp_board = self.merge_board_left(temp_board)
            temp_board = np.fliplr(temp_board)
            
        elif dir == 'up':
            temp_board = np.rot90(board_before_merge)
            temp_board = self.merge_board_left(temp_board)
            temp_board = np.rot90(temp_board, 3)
            
        elif dir == 'down':
            temp_board = np.rot90(board_before_merge, 3)
            temp_board = self.merge_board_left(temp_board)
            temp_board = np.rot90(temp_board)
        
        board_after_merge = temp_board
        
        # Check if any tiles moved
        check_same = board_after_merge == board_before_merge
        if check_same.all() == True:
            return board_after_merge
            
        else:
            board_after_merge_new_tile, status = self.new_tile(board_after_merge)
            return board_after_merge_new_tile
        
        #if status == 0:
        #    return board_after
        #elif status == -1:
            

    
    # Method for generating new game board
    def new_game_board(self):
        # initialize gameboard as an NxN-dimensional array 
        # filled with zeros and two non-zero initial tiles with value 2
        if self.custom_start == False:
            temp_board = np.zeros((self.dim, self.dim), dtype=int)
            
            x1_init, y1_init = np.random.randint(0,self.dim-1,1)[0], np.random.randint(0,self.dim-1,1)[0]
            x2_init, y2_init = np.random.randint(0,self.dim-1,1)[0], np.random.randint(0,self.dim-1,1)[0]
            
            while x2_init == x1_init and y2_init == y1_init:
                x2_init, y2_init = np.random.randint(0,self.dim-1,1)[0], np.random.randint(0,self.dim-1,1)[0]
            
            temp_board[x1_init][y1_init] += 2
            temp_board[x2_init][y2_init] += 2
            
            self.board = temp_board
           
        else:
            self.board = self.custom_board
            self.dim = len(self.custom_board)
        
    
    # Method for starting a new game
    def new_2048(self):
        #print('New game pressed')
        self.score = 0
        self.new_game_board()
        self.update_tiles()
        self.update_score()
        self.update_game_over()
        
    
    
          
      

# Welcome to my 2048 game!

This is my implementation of the classic game _2048_ by Gabriele Cirulli.

Select a board size and hit 'Play!'

The game will open in a new window. To select a new size, first close the current game window.

I hope you enjoy this game as much as I do!


In [1]:
game_sizes=[3 + i for i in range(8)]

if __name__ == '__main__':
    new_game = interact_manual(main, game_size=[(str(x)+' by '+str(x), x) for x in game_sizes]);
    new_game.widget.children[0].description = 'Board Size: '
    new_game.widget.children[1].description = 'Play!'
    new_game.widget.children[1].style.button_color = 'lightblue'

NameError: name 'interact_manual' is not defined